### Phase 1: Foundations & Simple Graphs (The State Machine)

##### 1. Understand the State

In [ ]:
from typing import TypedDict
# Recommended approach: Use a TypedDict for clear structure

class GraphState(TypedDict):
    user_input: str # The user's initial question
    summary_output: str  # The LLM's response or a summary
    tool_results: list  # A list to store tool results

# Initial state for a run:
initial_state = {"user_input": "What is LangGraph?", "summary_output": "", "tool_results": []}


In [ ]:
initial_state

##### 2. Learn Nodes (functions, LLM calls, tools)


In [ ]:
# 1. A pure Python function node
def log_input_node(state: GraphState) -> dict:
    print(f"User asked: {state['user_input']}")
    # This node doesn't need to change the state, so it returns an empty dict or the original state.
    return {} 

In [ ]:
def summarize_node(state: GraphState) -> dict:
    # Pretend an LLM or tool generated this text
    summary = f"LangGraph orchestrates how different nodes and tools work together."
    
    # Return a partial update to the shared state
    return {"summary_output": summary}


In [ ]:
from langgraph.graph import StateGraph

# Create the graph
graph = StateGraph(GraphState)

# Add both nodes
graph.add_node("log_input", log_input_node)
graph.add_node("summarize", summarize_node)

# Define the order
graph.set_entry_point("log_input")
graph.add_edge("log_input", "summarize")

# Compile it
app = graph.compile()

# Run it and see state updates
final_state = app.invoke(initial_state)

print("\n--- Final State ---")

print(final_state)


In [ ]:
# 2. An LLM node (using a LangChain runnable)
# The runnable itself is the node.
from langchain_ollama import ChatOllama
llm = ChatOllama(model="qwen3:0.6b")
# This LLM node will typically be configured to output to a specific state key later.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define your state
class GraphState(TypedDict):
    user_input: str
    summary_output: str

# Create the LLM and a simple prompt pipeline
llm = ChatOllama(model="qwen3:0.6b")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{question}")
])
chain = prompt | llm | StrOutputParser()

# Wrap the LLM chain in a node that reads state and returns a partial state update
def llm_node(state: GraphState) -> dict:
    try:
        answer = chain.invoke({"question": state["user_input"]})
    except Exception as e:
        answer = f"[LLM error: {e}]"
    return {"summary_output": answer}

# Create a graph
graph = StateGraph(GraphState)

# Add the node
graph.add_node("llm_node", llm_node)

# Set edges
graph.add_edge(START, "llm_node")
graph.add_edge("llm_node", END)

# Compile the graph
app = graph.compile()

# Run it
result = app.invoke({"user_input": "What is LangGraph?", "summary_output": ""})
print(result.get("summary_output", result))

### Note: Using an LLM as a node in a StateGraph

- LangGraph nodes should accept the shared state and return a partial state update (a dict to merge).
- Instead of passing the raw LLM as a node, wrap it in a function that reads `user_input` and returns `{"summary_output": ...}`.
- The simple prompt + `StrOutputParser` ensures the node returns a plain string for easy merging and printing.

##### 3. Learn Edges (data flow between nodes)

In [ ]:
from langgraph.graph import StateGraph, END

# Assume we have three nodes: node_a, node_b, and node_c
workflow = StateGraph(GraphState) 

# Add the nodes to the graph
workflow.add_node("step_1_input", log_input_node)
workflow.add_node("step_2_summary", summarize_node)

# 1. Edge from starting node to the summarizer
workflow.add_edge("step_1_input", "step_2_summary")

# 2. Edge from the summarizer to the final output (END)
workflow.add_edge("step_2_summary", END)

# Set the entry point
workflow.set_entry_point("step_1_input") 

# When you compile and run:
app = workflow.compile()
result = app.invoke(initial_state)


##### 4. Graph construction basics (StateGraph, add_node, add_edge)

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class SimpleState(TypedDict):
    count: int

def add_one(state: SimpleState):
    return {"count": state["count"] + 1}

# Build the graph
workflow = StateGraph(SimpleState)
workflow.add_node("A", add_one)
workflow.add_node("B", add_one) # Can reuse the function!

workflow.add_edge("A", "B") # A -> B
workflow.add_edge("B", END) # B -> END

workflow.set_entry_point("A") # Start at A
app = workflow.compile()

# Run: {count: 0} -> A makes it {count: 1} -> B makes it {count: 2}
result = app.invoke({"count": 0})
print(result) 
# Output: {'count': 2}


##### 5. Building the First Simple Graph

In [ ]:
# Phase 1: Pure Python Node
def take_input_node(input_dict: dict) -> dict:
    # In a real app, this would get input from a UI, here we assume it's passed directly
    return {"user_input": input_dict["user_input"], "summary_output": ""}

def save_output_node(state: GraphState) -> dict:
    final_summary = state["summary_output"].strip()
    print(f"\n--- FINAL SUMMARY ---\n{final_summary}")
    # Could save to a file/DB here.
    return {} # No state change needed, just an action


##### 6. Add an LLM Node

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize the LLM and the parser
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
parser = StrOutputParser()

# The function node that calls the LLM
def summarize_node(state: GraphState) -> dict:
    user_text = state["user_input"]
    prompt = f"Please summarize the following text concisely:\n\n---\n{user_text}"

    # Call the LLM
    summary = llm.invoke([HumanMessage(content=prompt)]).content

    # Return the update for the state
    return {"summary_output": summary}


##### 7. Connect nodes with sequential edges


In [ ]:
# Re-using nodes from above

# 1. Define the workflow structure
workflow = StateGraph(GraphState) 

# 2. Add all the nodes
workflow.add_node("input_handler", take_input_node)
workflow.add_node("llm_summarizer", summarize_node)
workflow.add_node("output_saver", save_output_node)

# 3. Connect them sequentially (The Assembly Line)
workflow.add_edge("input_handler", "llm_summarizer")
workflow.add_edge("llm_summarizer", "output_saver")
workflow.add_edge("output_saver", END)

# 4. Set the start and compile
workflow.set_entry_point("input_handler")
app = workflow.compile()


##### 8. Pass variables in the state dictionary

In [ ]:

# State update from take_input_node:
# returns {"user_input": "text here"}

# summarize_node reads it:
# user_text = state["user_input"] 

# summarize_node updates it:
# returns {"summary_output": "The summary..."}

# save_output_node reads it:
# final_summary = state["summary_output"] 

# three phases

In [2]:
# 🧩 Phase 1 – Define the State

from typing import TypedDict

# Define what data your graph will carry around
class GraphState(TypedDict):
    user_input: str
    summary_output: str

# ⚙️ Phase 2 – Create the Nodes

# 🟢 Node 1: Handle Input
def take_input_node(state: GraphState) -> dict:
    print(f"🟢 Got input: {state['user_input']}")
    # No change needed yet, just confirming we received input
    return {}

# 🧠 Node 2: LLM Summarizer
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

# Initialize the LLM
llm = ChatOllama(model="qwen3:0.6b")


def summarize_node(state: GraphState) -> dict:
    user_text = state["user_input"]
    prompt = f"Summarize this text briefly:\n\n{user_text}"
    summary = llm.invoke([HumanMessage(content=prompt)]).content
    print("🧠 LLM finished summarizing.")
    return {"summary_output": summary}

# 📦 Node 3: Output Saver

def save_output_node(state: GraphState) -> dict:
    print("\n--- FINAL SUMMARY ---")
    print(state["summary_output"])
    return {}


# 🔗 Phase 3 – Build and Run the Graph
from langgraph.graph import StateGraph, END

# Build workflow
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("input_handler", take_input_node)
workflow.add_node("llm_summarizer", summarize_node)
workflow.add_node("output_saver", save_output_node)

# Connect them in sequence
workflow.add_edge("input_handler", "llm_summarizer")
workflow.add_edge("llm_summarizer", "output_saver")
workflow.add_edge("output_saver", END)

# Set entry point and compile
workflow.set_entry_point("input_handler")
app = workflow.compile()



c:\Langgraph\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
result = app.invoke({"user_input": "LangGraph lets you connect Python logic and LLMs into agentic workflows."})
print("\n✅ Final state returned by LangGraph:")
print(result)


🟢 Got input: LangGraph lets you connect Python logic and LLMs into agentic workflows.
🧠 LLM finished summarizing.

--- FINAL SUMMARY ---
LangGraph combines Python logic with LLMs to create agentic workflows.

✅ Final state returned by LangGraph:
{'user_input': 'LangGraph lets you connect Python logic and LLMs into agentic workflows.', 'summary_output': 'LangGraph combines Python logic with LLMs to create agentic workflows.'}
